# Transformers

© Data Trainers LLC. GPL v 3.0.

Author: Axel Sirota


---



Inspired highly on the tutorial [NMT with Transformers](https://www.tensorflow.org/text/tutorials/transformer) which takes the code from the original Transformer model paper originally proposed in ["Attention is all you need"](https://arxiv.org/abs/1706.03762) by Vaswani et al. (2017).

## Prep

In [1]:
!pip install -U nltk 'gensim==4.2.0' 'keras-nlp' 'keras-preprocessing' 'tensorflow-text>=2.11'

In [2]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda, ELU, Conv1D, MaxPooling1D, Dropout
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras import preprocessing
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import Model, Input
import tensorflow_text as tf_text
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import numpy as np
import re
import random
import os
import pandas as pd
import gensim
import warnings
import nltk
import time

TRACE = False

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## The Transformer Layers

In this demo we will create, from scratch, with the same tools the original Authors had, the Transformer architecture. Why? To understand how it works, why it works, and exactly what is novel!

<table>
<tr>
  <th colspan=1>The original Transformer diagram</th>
  <th colspan=1>A representation of a 4-layer Transformer</th>
</tr>
<tr>
  <td>
   <img width=400 src="https://www.tensorflow.org/images/tutorials/transformer/transformer.png"/>
  </td>
  <td>
   <img width=307 src="https://www.tensorflow.org/images/tutorials/transformer/Transformer-4layer-compact.png"/>
  </td>
</tr>
</table>

Each of the components in these two diagrams will be explained as you progress through the demo.


### What did we have before?

Before, we used Cross Attention or self attention, remember? And for sequence data we basically used it like this:

<table>
<tr>
  <th colspan=1>Seq2Seq with attention</th>
<tr>
<tr>
  <td>
   <img src="https://www.dropbox.com/s/r6u7ll5nlt96t9f/seq2seq.png?raw=1"/>
  </td>
</tr>
</table>



Where we input attention with the hidden state to create another updated hidden state we could input into the next cell. And this worked well on medium sized sentences, but was hard to train and unstable. Now that we know this, the Transformer basicaly tried to get rid of the RNN by using **only** attention

### The embedding and positional encoding layer

The inputs to both the encoder and decoder use the same embedding and positional encoding logic.

<table>
<tr>
  <th colspan=1>The embedding and positional encoding layer</th>
<tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/PositionalEmbedding.png"/>
  </td>
</tr>
</table>

In [3]:
## This comes straight from the paper

def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1)

  return tf.cast(pos_encoding, dtype=tf.float32)

In [4]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x

In [5]:
pos = PositionalEmbedding(5000, 100)

In [6]:
input = tf.constant(np.random.randint(1,5000, size=(3,26)))
response = pos(input)
response.shape

TensorShape([3, 26, 100])

In [7]:
response._keras_mask

<tf.Tensor: shape=(3, 26), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True]])>

### Add and normalize

<table>
<tr>
  <th colspan=2>Add and normalize</th>
<tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/Add+Norm.png"/>
  </td>
</tr>
</table>

Note: Use `Add` layer instead of + to propagate masks

We will create a BaseAttention layer that inherits the Add+Norm and then each subclass of attention will implement the correct one

In [8]:
class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

### Self Attention layer

<table>
<tr>
  <th colspan=1>The global self attention layer</th>
<tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/SelfAttention.png"/>
  </td>
</tr>
</table>

In [9]:
class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    # We need to compare everything with everything, therefore Q, K and V must be the input
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    x = self.add([x, attn_output])  # This one comes from the base class
    x = self.layernorm(x)  # This one comes from the base class
    return x

Let's test it!

In [10]:
embedding_dim = 100
vocab_size = 5000
input = tf.constant(np.random.randint(1,vocab_size, size=(3,26)))

# First we apply the PositionalEmbedding to embed into what the attention layer expects
pos = PositionalEmbedding(vocab_size, embedding_dim)

# Then we do the self attention, the n_heads is arbitrary
gsa = GlobalSelfAttention(num_heads=3, key_dim=embedding_dim)


response = gsa(pos(input))
response.shape

TensorShape([3, 26, 100])

Notice the shape is the same, since MHA concats all 3 heads and the we add everything

### The cross attention layer

This layer connects the encoder and decoder. This layer is the most straight-forward use of attention in the model, it performs the same task as the attention block in the previous demo (and we will copy it).

<table>
<tr>
  <th colspan=1>The cross attention layer</th>
<tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/CrossAttention.png"/>
  </td>
</tr>
</table>

In [11]:
class CrossAttention(BaseAttention):
  def call(self, x, context):
    attn_output, attn_scores = self.mha(
        query=x,
        key=context,  # This is the key part!!
        value=context,  # This is the key part!!
        return_attention_scores=True)

    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [12]:
embedding_dim_es = 100
vocab_size_es = 5000

embedding_dim_en = 512
vocab_size_en = 6000

# We are supposing the model will translate Spanish to English, so context for CrossAttention will be the spanish input.

input_es = tf.constant(np.random.randint(1,vocab_size_es, size=(3,26)))
input_en = tf.constant(np.random.randint(1,vocab_size_es, size=(3,24)))


pos_es = PositionalEmbedding(vocab_size_es, embedding_dim_es)
pos_en = PositionalEmbedding(vocab_size_en, embedding_dim_en)


gsa = GlobalSelfAttention(num_heads=3, key_dim=embedding_dim_es)
cross = CrossAttention(num_heads=3, key_dim=embedding_dim_en)


context = gsa(pos_es(input_es)) # Forget about the feed forwards

response = cross(pos_en(input_en), context=context) # Forget about masked attention for now, assume it is the identity

response.shape

TensorShape([3, 24, 512])

Notice the shape is (batch_size, words in sentence in output, embedding_dim) , regardless the input sentence had more words or other embedding dim. We are doing a good move forward!

### The causal self attention layer (Masked Multi Headed Attention)

<table>
<tr>
  <th colspan=1>The causal self attention layer</th>
<tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/CausalSelfAttention.png"/>
  </td>
</tr>
</table>



The only big difference in the masked multi headedd attention is that we cannot attend to words in the future, so we will use a mask such that the `Nth` word can only see the first `N-1` words and not all the sentence.

In [13]:
class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)  # This is the key!
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

<table>
<tr>
  <th colspan=1>The causal self attention layer</th>
<tr>
<tr>
  <td>
   <img width=330 src="https://www.tensorflow.org/images/tutorials/transformer/CausalSelfAttention-new-full.png"/>
  </td>
</tr>
</table>

Notice in the diagram above how the query can onlly attend the values for the past

In [14]:
embedding_dim_en = 512
vocab_size_en = 6000

# We are supposing the model will translate Spanish to English, so context for CrossAttention will be the spanish input.

input_en = tf.constant(np.random.randint(1,vocab_size_es, size=(3,24)))


pos_en = PositionalEmbedding(vocab_size_en, embedding_dim_en)

csa = CausalSelfAttention(num_heads =3, key_dim=embedding_dim_en)

response = csa(pos_es(input_en))

response.shape

TensorShape([3, 24, 100])

### The feed forward network

The transformer also includes this point-wise feed-forward network in both the encoder and decoder:

<table>
<tr>
  <th colspan=1>The feed forward network</th>
<tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/FeedForward.png"/>
  </td>
</tr>
</table>

In [15]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x)
    return x


### The encoder layer

The encoder contains a stack of `N` encoder layers. Where each `EncoderLayer` contains a `GlobalSelfAttention` and `FeedForward` layer:

<table>
<tr>
  <th colspan=1>The encoder layer</th>
<tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/EncoderLayer.png"/>
  </td>
</tr>
</table>

In [16]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x

In [17]:
embedding_dim = 100
vocab_size = 5000
input = tf.constant(np.random.randint(1,vocab_size, size=(3,26)))
pos = PositionalEmbedding(vocab_size, embedding_dim)
sample_encoder_layer = EncoderLayer(d_model=embedding_dim, num_heads=3, dff=1012)
response = sample_encoder_layer(pos(input))
response.shape

TensorShape([3, 26, 100])

### The encoder

Notice we need to be able to repeat the past EncoderLayer Nx times, so we need another Layer that is able to do exactly that

<table>
<tr>
  <th colspan=1>The encoder</th>
<tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/Encoder.png"/>
  </td>
</tr>
</table>

In [18]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads,
               dff, vocab_size, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(
        vocab_size=vocab_size, d_model=d_model)

    self.enc_layers = [
        EncoderLayer(d_model=d_model,
                     num_heads=num_heads,
                     dff=dff,
                     dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    # `x` is token-IDs shape: (batch, seq_len)
    x = self.pos_embedding(x)  # Shape `(batch_size, seq_len, d_model)`.

    # Add dropout.
    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x)

    return x  # Shape `(batch_size, seq_len, d_model)`.

In [19]:
embedding_dim = 100
vocab_size = 5000
input = tf.constant(np.random.randint(1,vocab_size, size=(3,26)))
sample_encoder = Encoder(num_layers=4,
                         d_model=embedding_dim,
                         num_heads=3,
                         dff=512,
                         vocab_size=vocab_size)
response = sample_encoder(input)
response.shape

TensorShape([3, 26, 100])

We got our Encoder!! Yahoo!!

### The decoder layer

Same as before we need a Decoder layer that uses the Attention layers and then another layer to permit having Nx layers of decoding

<table>
<tr>
  <th colspan=1>The decoder layer</th>
<tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/DecoderLayer.png"/>
  </td>
</tr>
</table>

In [20]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1):
    super().__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.cross_attention = CrossAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context)

    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.cross_attention.last_attn_scores

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x

In [21]:
embedding_dim_es = 100
vocab_size_es = 5000

embedding_dim_en = 512
vocab_size_en = 6000

# We are supposing the model will translate Spanish to English, so context for CrossAttention will be the spanish input.

input_es = tf.constant(np.random.randint(1,vocab_size_es, size=(3,26)))
input_en = tf.constant(np.random.randint(1,vocab_size_es, size=(3,24)))

pos_en = PositionalEmbedding(vocab_size_en, embedding_dim_en)


encoder =  Encoder(num_layers=2, d_model=embedding_dim_es, num_heads=3, dff=512, vocab_size=vocab_size_es)

context = encoder(input_es)

decoder_layer = DecoderLayer(d_model=embedding_dim_en, num_heads=3, dff=218, dropout_rate=0.2)

response = decoder_layer(pos_en(input_en), context=context)

response.shape

TensorShape([3, 24, 512])

### The Decoder

Similar to the `Encoder`, the `Decoder` consists of a `PositionalEmbedding`, and a stack of `DecoderLayer`s:

<table>
<tr>
  <th colspan=1>The embedding and positional encoding layer</th>
<tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/Decoder.png"/>
  </td>
</tr>
</table>

In [22]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size,
                                             d_model=d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

    self.last_attn_scores = None

  def call(self, x, context):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x, context)

    self.last_attn_scores = self.dec_layers[-1].last_attn_scores

    # The shape of x is (batch_size, target_seq_len, d_model).
    return x

In [23]:
embedding_dim_es = 100
vocab_size_es = 5000

embedding_dim_en = 512
vocab_size_en = 6000

# We are supposing the model will translate Spanish to English, so context for CrossAttention will be the spanish input.

input_es = tf.constant(np.random.randint(1,vocab_size_es, size=(3,26)))
input_en = tf.constant(np.random.randint(1,vocab_size_es, size=(3,24)))

encoder =  Encoder(num_layers=2, d_model=embedding_dim_es, num_heads=3, dff=512, vocab_size=vocab_size_es)

context = encoder(input_es)

decoder = Decoder(num_layers=3, d_model=embedding_dim_en, num_heads=5, dff=124, vocab_size=vocab_size_en)

response = decoder(input_en, context=context)

response.shape

TensorShape([3, 24, 512])

## The Transformer Model

You now have `Encoder` and `Decoder`. To complete the `Transformer` model, you need to put them together and add a final linear (`Dense`) layer which converts the resulting vector at each location into output token probabilities.

The output of the decoder is the input to this final linear layer.

<table>
<tr>
  <th colspan=1>The transformer</th>
<tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/transformer.png"/>
  </td>
</tr>
</table>

In [24]:
class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
    super().__init__()
    self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=input_vocab_size,
                           dropout_rate=dropout_rate)

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    context, x  = inputs

    context = self.encoder(context)  # (batch_size, context_len, d_model)

    x = self.decoder(x, context)  # (batch_size, target_len, d_model)

    # Final linear layer output.
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    # Return the final output and the attention weights.
    return logits

In [25]:
embedding_dim = 100
vocab_size_es = 5000
vocab_size_en = 6000

num_layers = 4
dff = 512
num_heads = 8
dropout_rate = 0.1

# We are supposing the model will translate Spanish to English, so context for CrossAttention will be the spanish input.

input_es = tf.constant(np.random.randint(1,vocab_size_es, size=(3,26)))
input_en = tf.constant(np.random.randint(1,vocab_size_es, size=(3,24)))

transformer = Transformer(
    num_layers=num_layers,
    d_model=embedding_dim,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=vocab_size_es,
    target_vocab_size=vocab_size_en,
    dropout_rate=dropout_rate)

response = transformer((input_es, input_en))
response.shape

TensorShape([3, 24, 6000])

In [26]:
transformer.summary()


Model: "transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_3 (Encoder)         multiple                  2203648   
                                                                 
 decoder_1 (Decoder)         multiple                  3594448   
                                                                 
 dense_42 (Dense)            multiple                  606000    
                                                                 
Total params: 6404096 (24.43 MB)
Trainable params: 6404096 (24.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Download and prepare the dataset

The steps you need to take to prepare the data:

1. Add a *start* and *end* token to each sentence.
2. Clean the sentences by removing special characters.
3. Create a word index and reverse word index (dictionaries mapping from word → id and id → word).
4. Pad each sentence to a maximum length.


In [27]:
%%writefile get_data.sh
if [ ! -f spa.txt ]; then
  wget -O spa.txt https://www.dropbox.com/s/ke42pnpydmy6oa6/spa.txt?dl=0
fi

Overwriting get_data.sh


In [28]:
!bash get_data.sh

In [29]:
! head spa.txt

Go.	Ve.
Go.	Vete.
Go.	Vaya.
Go.	Váyase.
Hi.	Hola.
Run!	¡Corre!
Run.	Corred.
Who?	¿Quién?
Fire!	¡Fuego!
Fire!	¡Incendio!


In [30]:
def load_data(path):
  text = path.read_text(encoding='utf-8')

  lines = text.splitlines()
  pairs = [line.split('\t') for line in lines]

  context = np.array([context for target, context in pairs])
  target = np.array([target for target, context in pairs])

  return target, context

In [31]:
import pathlib
target_raw, context_raw = load_data(pathlib.Path('./spa.txt'))
print(context_raw[-1])

Si quieres sonar como un hablante nativo, debes estar dispuesto a practicar diciendo la misma frase una y otra vez de la misma manera en que un músico de banjo practica el mismo fraseo una y otra vez hasta que lo puedan tocar correctamente y en el tiempo esperado.


In [32]:
print(target_raw[-1])

If you want to sound like a native speaker, you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo.


In [33]:
BUFFER_SIZE = len(context_raw)
BATCH_SIZE = 256


is_train = np.random.uniform(size=(len(target_raw),)) < 0.8

train_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[is_train], target_raw[is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))
val_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[~is_train], target_raw[~is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))

In [34]:
for example_context_strings, example_target_strings in train_raw.take(1):
  print(example_context_strings[:5])
  print()
  print(example_target_strings[:5])
  break

tf.Tensor(
[b'Estoy muy orgulloso de esto.'
 b'\xc2\xbfSab\xc3\xadas que Tom y Mary eran primos?'
 b'Tom quit\xc3\xb3 la tapa de la caja.'
 b'\xc2\xbfPor qu\xc3\xa9 no est\xc3\xa1s comiendo?' b'La he visto antes.'], shape=(5,), dtype=string)

tf.Tensor(
[b"I'm really proud of this." b'Did you know Tom and Mary were cousins?'
 b'Tom removed the lid from the box.' b"Why aren't you eating?"
 b'I have seen her before.'], shape=(5,), dtype=string)


### Text processing

One of the goals of this tutorial is to build a model that can be exported as a `tf.saved_model`. To make that exported model useful it should take `tf.string` inputs, and return `tf.string` outputs: All the text processing happens inside the model. Mainly using a `layers.TextVectorization` layer.

In [35]:
example_text = tf.constant('¿Todavía está en casa?')

print(example_text.numpy())
print(tf_text.normalize_utf8(example_text, 'NFKD').numpy())

b'\xc2\xbfTodav\xc3\xada est\xc3\xa1 en casa?'
b'\xc2\xbfTodavi\xcc\x81a esta\xcc\x81 en casa?'


In [36]:
def tf_lower_and_split_punct(text):
  # Split accented characters.
  text = tf_text.normalize_utf8(text, 'NFKD')
  text = tf.strings.lower(text)
  # Keep space, a to z, and select punctuation.
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
  # Add spaces around punctuation.
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  # Strip whitespace.
  text = tf.strings.strip(text)

  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text

In [37]:
print(example_text.numpy().decode())
print(tf_lower_and_split_punct(example_text).numpy().decode())

¿Todavía está en casa?
[START] ¿ todavia esta en casa ? [END]


### Text Vectorization


Here comes the key part, using the layer TextVectorization we provide how to process text and how to construct the vocabulary, which we will later refer back

In [38]:
context_text_processor = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=vocab_size_es,
    ragged=True)

In [39]:
context_text_processor.adapt(train_raw.map(lambda context, target: context))

# Here are the first 10 words from the vocabulary:
context_text_processor.get_vocabulary()[:10]

['', '[UNK]', '[START]', '[END]', '.', 'que', 'de', 'el', 'a', 'no']

In [40]:
target_text_processor = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=vocab_size_en,
    ragged=True)

target_text_processor.adapt(train_raw.map(lambda context, target: target))
target_text_processor.get_vocabulary()[:10]

['', '[UNK]', '[START]', '[END]', '.', 'the', 'i', 'to', 'you', 'tom']

notice we passed to ids, not padded yet

In [41]:
example_tokens = context_text_processor(example_context_strings)
example_tokens[:3, :]

<tf.RaggedTensor [[2, 41, 42, 1040, 6, 57, 4, 3],
 [2, 13, 1863, 5, 10, 33, 32, 612, 2317, 12, 3],
 [2, 10, 1141, 11, 3357, 6, 11, 450, 4, 3]]>

In [42]:
def process_text(context, target):
  context = context_text_processor(context).to_tensor()
  target = target_text_processor(target)
  targ_in = target[:,:-1].to_tensor()
  targ_out = target[:,1:].to_tensor()
  return (context, targ_in), targ_out


train_ds = train_raw.map(process_text, tf.data.AUTOTUNE)
val_ds = val_raw.map(process_text, tf.data.AUTOTUNE)

## Training

In [43]:
for (ex_context_tok, ex_tar_in), ex_tar_out in train_ds.take(1):
  print(ex_context_tok[0, :10].numpy())
  print()
  print(ex_tar_in[0, :10].numpy())
  print(ex_tar_out[0, :10].numpy())

[  2  13  21   5 196  91  12   3   0   0]

[ 2 78 20 16 98 11  0  0  0  0]
[78 20 16 98 11  3  0  0  0  0]


In [44]:
logits = transformer((ex_context_tok, ex_tar_in))

print(f'Context tokens, shape: (batch, s, units) {ex_context_tok.shape}')
print(f'Target tokens, shape: (batch, t) {ex_tar_in.shape}')
print(f'logits, shape: (batch, t, target_vocabulary_size) {logits.shape}')

Context tokens, shape: (batch, s, units) (256, 32)
Target tokens, shape: (batch, t) (256, 30)
logits, shape: (batch, t, target_vocabulary_size) (256, 30, 6000)


In [45]:
def masked_loss(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    loss = loss_fn(y_true, y_pred)

    # Mask off the losses on padding.
    mask = tf.cast(y_true != 0, loss.dtype)
    loss *= mask

    # Return the total.
    return tf.reduce_sum(loss)/tf.reduce_sum(mask)

In [46]:
def masked_acc(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)

    match = tf.cast(y_true == y_pred, tf.float32)
    mask = tf.cast(y_true != 0, tf.float32)

    return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [47]:
transformer.compile(optimizer='adam',
              loss=masked_loss,
              metrics=[masked_acc, masked_loss])

In [ ]:
history = transformer.fit(
    train_ds,
    epochs=25,
    steps_per_epoch=50,
    validation_data=val_ds,
    validation_steps=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3)])

Epoch 1/25
50/50 [==============================] - 166s 3s/step - loss: 6.4545 - masked_acc: 0.1924 - masked_loss: 6.4519 - val_loss: 5.2232 - val_masked_acc: 0.2492 - val_masked_loss: 5.2233
Epoch 2/25
17/50 [=========>....................] - ETA: 1:24 - loss: 5.0945 - masked_acc: 0.2494 - masked_loss: 5.0949

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('CE/token')
plt.legend()

In [ ]:
plt.plot(history.history['masked_acc'], label='accuracy')
plt.plot(history.history['val_masked_acc'], label='val_accuracy')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('CE/token')
plt.legend()

In [ ]:
context_text_processor('Hola')

## Translate

In [ ]:
class Translator(tf.Module):
  def __init__(self, context_text_processor, target_text_processor, transformer, max_tokens=50):
    self.context_text_processor = context_text_processor
    self.target_text_processor = target_text_processor
    self.index2word = { i : word for i, word in enumerate(target_text_processor.get_vocabulary())}
    self.transformer = transformer
    self.max_tokens = max_tokens

  def __call__(self, sentence):

    sentence = context_text_processor(sentence)[tf.newaxis]

    encoder_input = sentence


    # As the output language is English, initialize the output with the
    # Spanish `[START]` token.
    start = encoder_input[0][0][tf.newaxis]
    end = encoder_input[0][-1][tf.newaxis]
    # `tf.TensorArray` is required here (instead of a Python list), so that the
    # dynamic-loop can be traced by `tf.function`.
    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)

    for i in tf.range(self.max_tokens):
      output = tf.transpose(output_array.stack())
      predictions = self.transformer((encoder_input, output), training=False)

      # Select the last token from the `seq_len` dimension.
      predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`.

      predicted_id = tf.argmax(predictions, axis=-1)

      # Concatenate the `predicted_id` to the output which is given to the
      # decoder as its input.
      output_array = output_array.write(i+1, predicted_id[0])

      if predicted_id == end:
        break

    output = tf.transpose(output_array.stack())
    translated_text = ' '.join([ index2word.get(token) for token in tokens[0].numpy()])

    # `tf.function` prevents us from using the attention_weights that were
    # calculated on the last iteration of the loop.
    # So, recalculate them outside the loop.
    # self.transformer([encoder_input, output[:,:-1]], training=False)
    # attention_weights = self.transformer.decoder.last_attn_scores

    return output, translated_text

In [ ]:
translator = Translator(transformer=transformer, context_text_processor=context_text_processor, target_text_processor=target_text_processor)

In [ ]:
def print_translation(sentence, tokens, translated_text, ground_truth):
  print(f'{"Input:":15s}: {sentence}')
  print(f'{"Prediction":15s}: {tokens.numpy()}')
  print(f'{"Prediction":15s}: {translated_text}')
  print(f'{"Ground truth":15s}: {ground_truth}')

Example 1:

In [ ]:
sentence = 'Si me quieren encontrar, busquen en el cielo del oeste'
ground_truth = 'If you want to find me, look for the western sky'

tokens, translated_text = translator(
    sentence)
print_translation(sentence, tokens, translated_text, ground_truth)